In [1]:
include("..\\src\\ImportanceSampling.jl")
using ImportanceSampling
using Distributions

In [2]:
srand(2016);

In [3]:
# defined by θ (shape), not λ (rate)
struct MvExponential <: ContinuousMultivariateDistribution
    Es::Vector{Exponential{Float64}}

    function MvExponential(θs::Vector{<:Real})
        all(θs .> 0) || error("all θs must be > 0")
    
        return new(Exponential.(float(θs)))
    end
end

MvExponential(n::Integer) = MvExponential(ones(n))
Base.length(d::MvExponential) = length(d.Es)
Distributions._rand!(d::MvExponential, x::AbstractVector{<:Real}) = 
    x .= rand.(d.Es)
Distributions._logpdf(d::MvExponential, x::AbstractVector{<:Real}) = 
    sum(logpdf(d.Es[i], x[i]) for i in 1:length(d))

In [4]:
function f!(p, x)
    E1 = x[1]
    E2 = E1 + x[2]
    E3 = E1 + x[3]
    E4 = E2 + x[4]
    E5 = E2 + x[5]
    E6 = E3 + x[6]
    E7 = E3 + x[7]
    E8 = E3 + x[8]
    E9 = max(E5, E6, E7) + x[9]
    S10 = max(E4, E8, E9)
    p[1] = min(1, exp(-(70-S10)/θs[10]))
end

f! (generic function with 1 method)

In [5]:
θs = [4, 4, 2, 5, 2, 3, 2, 3, 2, 2]
critpath = zeros(10)
critpath[[1, 2, 4, 10]] = 1
p = MvExponential(θs)
q = MixtureDistribution([p, MvExponential((1+2*critpath).*θs), 
        MvExponential((1+4*critpath).*θs)], [1, 8, 1])

is = ImportanceSampler(f!, 1, q, p=p)
cvis = CvImportanceSampler(f!, 1, q, p=p, use_q=true)
;

In [6]:
X = rand(q, 10_000)

update!(is, X=X)
updateβ!(cvis, X=X[:, 1:1_000])
updateμ!(cvis, X=X[:, 1_001:end])
;

In [7]:
mean(is), sqrt(var(is)), ne(is), neσ(is), neγ(is)

(3.284965674331318e-5, 1.8768194445446077e-6, 2259.3479163406782, 1253.3942395102931, 1388.3378334786705)

In [8]:
mean(cvis), sqrt(var(cvis)), ne(cvis), neσ(cvis), neγ(cvis)

(3.274997774649563e-5, 1.98176783314659e-6, 2052.715757674475, 1143.7669241576093, 1265.7974644517128)

In [9]:
coeffs(cvis)

3×1 Array{Float64,2}:
 -0.000318414
 -0.00247881 
 -0.000305176